# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('amazon_baby.sframe')

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [5]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [6]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [7]:
len(giraffe_reviews)

785

In [8]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [9]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [10]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


# Define what is positive and negative sentiment

In [11]:
products['rating'].show()

Materializing SArray

In [12]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [13]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [14]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0,'recommend': 1.0, ...",1


In [15]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [16]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [17]:
def count_word(text,word):
    """
    Function that takes the text and split it into word
    and counts the number of occurence of that word
    input: text and word
    output: number of times the word appears
    """
    answer = text.split(" ")
    count = 0
    for occurence in answer:
        if word == occurence:
            count = count + 1
    return count

#sentence = "To be a programmer you need to have a sharp thinking brain"
#word_count = "a"
#print(sentence.split(" "))
#print(count_word(sentence,word_count))


In [18]:
selected_words[1]

'great'

In [39]:
for i in enumerate(selected_words):
    #print(i[1])
    products[i[1]] = products.apply(lambda row: count_word (row['review'], i[1]))
    print(i[1])
    print(sum(products[i[1]]))

awesome
1606
great
34509
fantastic
754
amazing
1114
love
32129
horrible
567
bad
3093
terrible
587
awful
303
wow
48
hate
945


In [20]:
products['sum'] = products.apply(lambda row: row['awesome']+
                                 row['great']+
                                 row['fantastic']+
                                 row['amazing']+
                                 row['love']+
                                 row['horrible']+
                                 row['bad']+
                                 row['terrible']+ 
                                 row['awful']+
                                 row['wow']+
                                 row['hate'])

In [37]:
sum(products['love'])

32129

In [24]:
train_data,test_data = products.random_split(.8,seed=0)

In [41]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 1.531736     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 1.759574     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 2.101447     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 3.446691     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [42]:
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.211327     | 0.843257          | 0.841821            |

| 5         | 6        | 0.255634     | 0.843257          | 0.841821            |

| 6         | 7        | 0.297517     | 0.843257          | 0.841821            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [65]:

df = selected_words_model.coefficients
df.print_rows(num_rows=12) 

+-------------+-------+-------+----------------------+----------------------+
|     name    | index | class |        value         |        stderr        |
+-------------+-------+-------+----------------------+----------------------+
| (intercept) |  None |   1   |  1.4508720017816794  | 0.008438478585588051 |
|   awesome   |  None |   1   |  0.8752716575545733  | 0.11388296333725204  |
|    great    |  None |   1   |   0.74283439114473   | 0.022531705253867895 |
|  fantastic  |  None |   1   |  0.7484721044248996  | 0.15500133746022535  |
|   amazing   |  None |   1   |  0.8774493635313501  | 0.13598807114947042  |
|     love    |  None |   1   |  1.3159105199754877  | 0.030766002897234725 |
|   horrible  |  None |   1   | -1.8649709697226922  | 0.10145679016304056  |
|     bad     |  None |   1   | -0.9452376618367807  | 0.043550940185266535 |
|   terrible  |  None |   1   | -1.9359996801606463  | 0.09941895298071148  |
|    awful    |  None |   1   | -1.6819776853094892  | 0.1386715

In [45]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8418208023060293,
 'auc': 0.6334050888535089,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  141  |
 |      0       |        1        |  5187 |
 |      1       |        1        | 27895 |
 |      1       |        0        |   81  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9137213796717875,
 'log_loss': 0.41439735681787504,
 'precision': 0.8432077867118071,
 'recall': 0.9971046611381184,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328

In [61]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9258242975424673,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 |      1       |        0        |  1344 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187240904,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0  

# Apply the sentiment classifier to better understand the Giraffe reviews

In [47]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [48]:
products['predicted_select'] = selected_words_model.predict(products, output_type = 'probability')

In [49]:
products

name,review,rating,word_count,sentiment,sum,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ...",1,1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ...",1,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ...",1,2,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0,'recommend': 1.0, ...",1,2,0


In [50]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [51]:
diaper_champ_reviews

name,review,rating,word_count,sentiment,sum,awesome
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1.0,'can': 1.0, 'you': 1.0, ...",1,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'try': 1.0, 'going':1.0, 're': 1.0, 'leav ...",0,0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",1,0,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",1,1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1.0,'especially': 1.0, ...",1,1,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1.0, 'huge': 1.0,'special': 1.0, 'good': ...",1,1,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'clean': 1.0, 'it': 8.0,'thing': 1.0, 'years': ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0, 'worth':1.0, 'other': 2.0, ...",1,0,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ...",1,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'need': 1.0, 'garbage':1.0, 'regular': 1.0, ...",1,2,0


# Sort the Giraffe reviews according to predicted sentiment

In [52]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [53]:
diaper_champ_reviews

name,review,rating,word_count,sentiment,sum,awesome
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",1,0,0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'parent': 1.0, 'makes': ...",1,1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0, 'worth':1.0, 'other': 2.0, ...",1,0,0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'bulk':1.0, 'any': 1.0, 't': ...",1,0,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'be': 1.0, 'won': 1.0,'open': 1.0, 'if': 1.0, ...",1,0,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'helps': 1.0, 'slower':1.0, 'changings': 1.0, ...",1,1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'looking':1.0, 'product': 1.0, ...",1,1,0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",1,1,0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'seems':1.0, 'all': 1.0, ...",1,0,0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'sell': 1.0, 'not': 2.0, ...",1,1,0


In [54]:
diaper_champ_reviews.tail()

name,review,rating,word_count,sentiment,sum,awesome
Baby Trend Diaper Champ,"Like other reviewers, Iput this diaper pail on ...",5.0,"{'recycled': 1.0,'plain': 1.0, 'money': ...",1,0,0
Baby Trend Diaper Champ,This product is nice intheory but not all that ...,1.0,"{'a': 1.0, 'else': 1.0,'buy': 1.0, 'save': 1.0, ...",0,0,0
Baby Trend Diaper Champ,I am really disappointedwith the Diaper Champ. ...,2.0,"{'pails': 1.0, 'look':1.0, 'will': 1.0, 'ba ...",0,1,0
Baby Trend Diaper Champ,My 8 year old yellow labwas able to get the top ...,1.0,"{'prevent': 1.0,'literature': 1.0, ...",0,1,0
Baby Trend Diaper Champ,"Well, the first threemonths were odor free!! ...",2.0,"{'have': 1.0, 'don': 1.0,'smell': 1.0, 'purcha ...",0,0,0
Baby Trend Diaper Champ,This is the worst diaperpail ever! It was great ...,1.0,"{'not': 1.0, 'does': 1.0,'well': 1.0, 'hopeful': ...",0,1,0
Baby Trend Diaper Champ,This thing freakin'stinks. Literally. I ...,1.0,"{'them': 1.0, 'to': 1.0,'listened': 1.0, 'have': ...",0,0,0
Baby Trend Diaper Champ,Worst diaper pale ever!!I've had mine for 2 y ...,1.0,"{'this': 1.0,'unexpectedly': 1.0, ...",0,0,0
Baby Trend Diaper Champ,I LOVED my Champ for thefirst year as it was all ...,2.0,"{'do': 1.0, 'fails': 1.0,'replace': 1.0, ...",0,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'try': 1.0, 'going':1.0, 're': 1.0, 'leav ...",0,0,0


## Show the most positive reviews

In [55]:
diaper_champ_reviews[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."

In [56]:
diaper_champ_reviews[1]['review']

'I have never written a review for Amazon but I saw some of the poor reviews on the Diaper Champ and had to add my two cents . . .When my sister got pregnant she could not decide if she should register for the Diaper Champ or the Diaper Genie.  She left it up to me and after reviewing both items I went with the Diaper Champ . . .  The idea of having to continually purchase bags for the Diaper Genie seemed like a waste when the Diaper Champ lets you use any type of bag . . .  My nephew is now over two years old and while he has begun potty training, my sister still uses the Diaper Champ I gave her . . .When it came time for me to choose a diaper disposal system, I put the Diaper Champ on my registry.  Yes, it can be a bit tricky to open (I found using two hands works best) but then again do I want something that is holding wet and otherwise dirty diapers to open easily?  True, the mechanism sometimes won\'t "swing" the dirty diaper into the pail but HELLO that is a sign that it is proba

# Most negative reivews

In [57]:
diaper_champ_reviews[-1]['review']

'My husband and I selected the Diaper "Champ" mainly because you can use ordinary trash bags and not be roped into buying the specialty refill bags, and it was moderately priced (a little less than the Diaper Dekor). It also seemed that the reviews of this product were generally more positive...The positives are:1. You can use any trash bag2. Easy to use and refillThe negatives are:1. The bag doesn\'t seal around the dirty diapers, so when it comes time to refill the bag, it\'s just like opening a regular trash can. Smells like the Champ is trying to knock YOU out with odor!2. The plastic seems to smell, ie. You put a dirty diaper in the hole, and flip the handle to dump the diaper into the champ. That "side" of the plastic dumper-thingie is in contact with the air inside the dirty diaper changer, so when you flip it over the next time to dispose of another diaper, you smell the last 8 diapers you put in there...pretty gross.3. The "odor seal" (some soft material) really seems to retai

In [58]:
diaper_champ_reviews[-2]['review']

"I LOVED my Champ for the first year as it was all I expected in ease, cleanliness, odor free with regular bag changes and the economical use of kitchen bags.  Then the liner began to come loose and get stuck in the tumbler and eventually had to be removed altogether making things a bit less odor resistant, but still manageable with regular bag changes, cleanings, airings and sprinklings of baking soda.  The real problem we have now is that no amount of cleaning and deoderizing the Champ after a bag change (about twice a week for us) has managed to eliminate the strong odor that seems to be clinging to the actual pail now.  We've tried a number of disinfectants, deoderizers and air freshners, but they are no match for the offensive odor of the Champ even when clean and empty.  We are so disappointed in the short-term success of this product.  It has been removed from my son's room and we are now taking each diaper out individually and looking for another pail.  I am really interested i

In [60]:
sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.9999999999895941]

In [ ]:
Out of the 11 words in selected_words, which one got the most negative weight in the selected_words_model?
(Tip: when printing the list of coefficients, make sure to use print_rows(rows=12) to print ALL coefficients.)

0 / 1 ponto


How do you compare the different learned models with the baseline approach where we are just predicting the majority class?   

